In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [58]:
def clean_df_to_games(df):
    team_stats = df.loc[df['TEAM_STATS_OR_NOT'] == 1].copy() # GETTING JUST THE TOTAL TEAM BOX SCORE STATS
    team_stats.drop(columns=['TEAM', 'TIME', 'DATE+TIME', 'B_+/-', 'USG_PCT', '+/-', 'TEAM_STATS_OR_NOT'], inplace=True) #DROPING IRRELEVANT COLUMNS FOR LATER IN THE SCRAPE
    home = team_stats.loc[team_stats['HOME'] == 1] # GETTING THE HOME DATAFRAME
    away = team_stats.loc[team_stats['HOME'] == 0] # GETTING THE AWAY DATAFRAME
    games = pd.merge(home,away, how='outer', on='GAMEID').copy() # MERGING THE TWO AND CREATING A DATAFRAME COPY
    new_col_names = []
    for n in games.columns: # FOR LOOP MEANT TO LABEL WHICH COLUMN IS HOME AND WHICH IS AWAY
        if '_x' in n:
            n = n.replace('_x','_HOME')
            new_col_names.append(n)
        elif '_y' in n:
            n = n.replace('_y','_AWAY')
            new_col_names.append(n)
        else:
            new_col_names.append(n)
    new_col_names
    games.columns = new_col_names # INSERTING THE COLUMN LABELS
    games['PLAYER_HOME'] = games['PLAYER_HOME'].map(lambda x:x[13:16]) # GETTING JUST THE TEAM NAMES
    games['PLAYER_AWAY'] = games['PLAYER_AWAY'].map(lambda x:x[13:16])
    games.rename(columns={'PLAYER_HOME':'TEAM_HOME','PLAYER_AWAY':'TEAM_AWAY','DATE_HOME':'DATE'}, inplace=True)
    games['TEAM_AWAY'] = ['CHA' if i == 'CHO' else i for i in games['TEAM_AWAY']] # REPLACING 'CHO' WITH 'CHA' SINCE THEY ARE THE SAME TEAM, JUST UNDERWENT A NAME-CHANGE
    games['TEAM_HOME'] = ['CHA' if i == 'CHO' else i for i in games['TEAM_HOME']]
    games['DATE'] = pd.to_datetime(games['DATE']) # TURNING THE DATE COLUMN INTO DATETIME DATA TYPE
    games.drop(columns = ['OPPONENT_AWAY', 'OPPONENT_HOME', 'SEASON_AWAY'], inplace=True) # DROPPING UNNECCESSARY OPPONENT COLUMNS
    games['OTs'] = games['MP_HOME'].map(lambda x:int((x-240)/25)) # MAKING OT'S COLUMN FROM MP_HOME
    games.drop(columns = ['MP_HOME', 'MP_AWAY'], inplace=True) # DROPPING IRRELEVANT MP HOME COLUMNS
    games.drop(columns=['HOME_HOME','HOME_AWAY','AWAY_HOME','AWAY_AWAY'], inplace=True) # DROPPING ADDITIONAL USELESS COLUMNS PREVIOUSLY INDICATING HOME AND AWAY
    games.rename(columns={'SEASON_HOME':'SEASON'}, inplace=True) # RENAMING THE SEASON_HOME COLUMN JUST SEASON
    games.drop('DATE_AWAY', axis=1,inplace=True) # DROPPING SECOND DATE COLUMN
    games['FG(3)_MISSED_HOME'] = games['FGA(3)_HOME'] - games['FG(3)_HOME'] # MAKING NEW FG(3) MISSED FEATURE
    games['FG(3)_MISSED_AWAY'] = games['FGA(3)_AWAY'] - games['FG(3)_AWAY'] 
    games['FG_MISSED_HOME'] = games['FGA_HOME'] - games['FG_HOME'] # MAKING NEW FG MISSED FEATURE
    games['FG_MISSED_AWAY'] = games['FGA_AWAY'] - games['FG_AWAY']
    games['BLOCKED_ATTEMPTS_HOME'] = games['BLK_AWAY'] # MAKING NEW BLOCKED ATTEMPTS FEATURE
    games['BLOCKED_ATTEMPTS_AWAY'] = games['BLK_HOME']
    games['TOV_FORCED_HOME'] = games['TOV_AWAY'] # MAKING NEW TURNOVERS FORCED FEATURE
    games['TOV_FORCED_AWAY'] = games['TOV_HOME']
    games['FG(2)_HOME'] = games['FG_HOME']-games['FG(3)_HOME'] # MAKING NEW FG(2) MAKES FEATURE
    games['FG(2)_AWAY'] = games['FG_AWAY']-games['FG(3)_AWAY']
    games['FGA(2)_HOME'] = games['FGA_HOME']-games['FGA(3)_HOME'] # MAKING NEW FG(2) ATTEMPTS FEATURE
    games['FGA(2)_AWAY'] = games['FGA_AWAY']-games['FGA(3)_AWAY']
    games['FG(2)_MISSED_HOME'] = games['FG_MISSED_HOME']-games['FG(3)_MISSED_HOME'] # MAKING NEW FG(2) MISSED FEATURE
    games['FG(2)_MISSED_AWAY'] = games['FG_MISSED_AWAY']-games['FG(3)_MISSED_AWAY']
    games.rename(columns={'WINS_AWAY':'WINS_RECORD_AWAY', 'LOSSES_AWAY':'LOSSES_RECORD_AWAY', 'WINS_HOME':'WINS_RECORD_HOME',
                           'LOSSES_HOME':'LOSSES_RECORD_HOME'}, inplace=True) # RENAMING TOTAL WINS AND TOTAL LOSSES COLUMNS FOR CLARITY
    team_cols = []
    for col in games.columns: # GETTING COLUMN NAMES WITHOUT HOME AWAY LABELS
        if '_HOME' in col:
            col = col.replace('_HOME', '') # GETTING  COLUMN NAMES
            team_cols.append(col)
    gen_cols = []
    for col in games.columns:
        if '_HOME' not in col:
            if '_AWAY' not in col:
                gen_cols.append(col) # GETTING THE GENERAL COLUMNS WITHOUT AWAY OR HOME
    team_cols_each = [] # GETTING BOTH HOME AWAY COLUMNS IN THE SAME LIST
    for col in team_cols:
        home = col+'_HOME'
        away = col+'_AWAY'
        team_cols_each.append(home)
        team_cols_each.append(away)
    games = games[team_cols_each+gen_cols]
    return games

In [59]:
df_16 = pd.read_csv('./raw_data/2016_2017_SEASONS.csv')
df_18 = pd.read_csv('./raw_data/2018_2019_SEASONS.csv')
df_14 = pd.read_csv('./raw_data/2014_2015_SEASONS.csv')

In [60]:
df_14.drop('Unnamed: 0', axis=1, inplace=True)
df_16.drop('Unnamed: 0', axis=1, inplace=True)
df_18.drop('Unnamed: 0', axis=1, inplace=True)

In [61]:
games_16 = clean_df_to_games(df_16)

In [62]:
games_16

,TEAM_HOME,TEAM_AWAY,FG_HOME,FG_AWAY,FGA_HOME,FGA_AWAY,FG_PCT_HOME,FG_PCT_AWAY,FG(3)_HOME,FG(3)_AWAY,...,FG(2)_HOME,FG(2)_AWAY,FGA(2)_HOME,FGA(2)_AWAY,FG(2)_MISSED_HOME,FG(2)_MISSED_AWAY,GAMEID,DATE,SEASON,OTs
0,CHI,CLE,37,38,87,94,0.425,0.404,7,9,...,30,29,68,65,38,36,201510270CHI,2015-10-27,2016,0
1,ATL,DET,37,37,82,96,0.451,0.385,8,12,...,29,25,55,67,26,42,201510270ATL,2015-10-27,2016,0
2,GSW,NOP,41,35,96,83,0.427,0.422,9,6,...,32,29,66,65,34,36,201510270GSW,2015-10-27,2016,0
3,ORL,WAS,37,33,100,84,0.370,0.393,5,7,...,32,26,74,56,42,30,201510280ORL,2015-10-28,2016,0
4,BOS,PHI,39,34,85,83,0.459,0.410,8,7,...,31,27,61,61,30,34,201510280BOS,2015-10-28,2016,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,ORL,DET,42,41,87,106,0.483,0.387,13,11,...,29,30,62,70,33,40,201704120ORL,2017-04-12,2017,0
2456,UTA,SAS,39,39,77,86,0.506,0.453,10,8,...,29,31,51,65,22,34,201704120UTA,2017-04-12,2017,0
2457,GSW,LAL,41,41,90,93,0.456,0.441,12,4,...,29,37,54,76,25,39,201704120GSW,2017-04-12,2017,0
2458,LAC,SAC,43,40,83,81,0.518,0.494,10,5,...,33,35,55,70,22,35,201704120LAC,2017-04-12,2017,0


In [63]:
games_16.to_csv('./cleaned_data/team_data/cleaned_games_16_17.csv')

In [64]:
games_18 = clean_df_to_games(df_18)

In [65]:
games_18.to_csv('./cleaned_data/team_data/cleaned_games_18_19.csv')

In [66]:
games_14 = clean_df_to_games(df_14)

In [67]:
games_14.to_csv('./cleaned_data/team_data/cleaned_games_14_15.csv')

In [68]:
games_14

,TEAM_HOME,TEAM_AWAY,FG_HOME,FG_AWAY,FGA_HOME,FGA_AWAY,FG_PCT_HOME,FG_PCT_AWAY,FG(3)_HOME,FG(3)_AWAY,...,FG(2)_HOME,FG(2)_AWAY,FGA(2)_HOME,FGA(2)_AWAY,FG(2)_MISSED_HOME,FG(2)_MISSED_AWAY,GAMEID,DATE,SEASON,OTs
0,IND,ORL,34,36,71,93,0.479,0.387,7,9,...,27,27,54,74,27,47,201310290IND,2013-10-29,2014,0
1,MIA,CHI,37,35,72,83,0.514,0.422,11,7,...,26,28,52,57,26,29,201310290MIA,2013-10-29,2014,0
2,LAL,LAC,42,41,93,83,0.452,0.494,14,8,...,28,33,64,62,36,29,201310290LAL,2013-10-29,2014,0
3,CLE,BRK,35,33,84,82,0.417,0.402,5,9,...,30,24,69,58,39,34,201310300CLE,2013-10-30,2014,0
4,TOR,BOS,38,32,86,66,0.442,0.485,5,3,...,33,29,69,53,36,24,201310300TOR,2013-10-30,2014,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,NYK,DET,33,41,77,81,0.429,0.506,7,10,...,26,31,58,55,32,24,201504150NYK,2015-04-15,2015,0
2456,PHI,MIA,38,39,83,89,0.458,0.438,5,12,...,33,27,54,58,21,31,201504150PHI,2015-04-15,2015,0
2457,MEM,IND,36,32,76,78,0.474,0.410,2,8,...,34,24,67,54,33,30,201504150MEM,2015-04-15,2015,0
2458,GSW,DEN,47,45,90,83,0.522,0.542,13,6,...,34,39,62,65,28,26,201504150GSW,2015-04-15,2015,0


# CLEANING PLAYER DATAFRAMES

In [69]:
players = df_14.loc[df_14['TEAM_STATS_OR_NOT'] == 0].copy()

In [70]:
players

,PLAYER,MP,FG,FGA,FG_PCT,FG(3),FGA(3),FG_PCT(3),FT,FTA,...,WINS,LOSSES,WIN_PCT,GAME_NO,TIME,DATE+TIME,WIN,LOSS,TEAM_STATS_OR_NOT,SEASON
0,"George,Paul",36.083333,8,16,0.500,3,6,0.500,5,7,...,1.0,0.0,1.000000,1.0,7:00p,2013-10-29 19:00:00,1,0,0,2014
1,"Stephenson,Lance",35.633333,8,12,0.667,2,3,0.667,1,2,...,1.0,0.0,1.000000,1.0,7:00p,2013-10-29 19:00:00,1,0,0,2014
2,"West,David",30.716667,4,11,0.364,0,0,NaN,5,5,...,1.0,0.0,1.000000,1.0,7:00p,2013-10-29 19:00:00,1,0,0,2014
3,"Hill,George",29.066667,2,8,0.250,0,2,0.000,3,4,...,1.0,0.0,1.000000,1.0,7:00p,2013-10-29 19:00:00,1,0,0,2014
4,"Hibbert,Roy",26.900000,3,6,0.500,0,0,NaN,2,4,...,1.0,0.0,1.000000,1.0,7:00p,2013-10-29 19:00:00,1,0,0,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56505,"Thompson,Jason",24.733333,5,8,0.625,0,0,NaN,2,4,...,29.0,53.0,0.353659,82.0,10:30p,2015-04-15 22:30:00,0,1,0,2015
56506,"Williams,Derrick",33.583333,9,16,0.563,2,4,0.500,2,4,...,29.0,53.0,0.353659,82.0,10:30p,2015-04-15 22:30:00,0,1,0,2015
56507,"Hollins,Ryan",23.266667,5,7,0.714,0,0,NaN,2,7,...,29.0,53.0,0.353659,82.0,10:30p,2015-04-15 22:30:00,0,1,0,2015
56508,"Stockton,David",19.383333,2,3,0.667,1,1,1.000,0,1,...,29.0,53.0,0.353659,82.0,10:30p,2015-04-15 22:30:00,0,1,0,2015


In [71]:
players.columns

Index(['PLAYER', 'MP', 'FG', 'FGA', 'FG_PCT', 'FG(3)', 'FGA(3)', 'FG_PCT(3)',
       'FT', 'FTA', 'FT_PCT', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV',
       'PF', 'PTS', '+/-', 'TS_PCT', 'EFG_PCT', '3PA_R', 'FT_R', 'ORB_PCT',
       'DRB_PCT', 'TRB_PCT', 'AST_PCT', 'STL_PCT', 'BLK_PCT', 'TOV_PCT',
       'USG_PCT', 'O_RTG', 'D_RTG', 'B_+/-', 'GAMEID', 'TEAM', 'HOME', 'AWAY',
       'OPPONENT', 'DATE', 'WINS', 'LOSSES', 'WIN_PCT', 'GAME_NO', 'TIME',
       'DATE+TIME', 'WIN', 'LOSS', 'TEAM_STATS_OR_NOT', 'SEASON'],
      dtype='object')

In [72]:
pct_cols = []
for col in players.columns:
    if 'PCT' in col:
        pct_cols.append(col)

In [73]:
pct_cols

['FG_PCT',
 'FG_PCT(3)',
 'FT_PCT',
 'TS_PCT',
 'EFG_PCT',
 'ORB_PCT',
 'DRB_PCT',
 'TRB_PCT',
 'AST_PCT',
 'STL_PCT',
 'BLK_PCT',
 'TOV_PCT',
 'USG_PCT',
 'WIN_PCT']

In [74]:
for i in pct_cols:
    print(players[i].isna().sum())

2586
19848
21795
2239
2586
0
0
0
0
0
0
1766
0
0


My hypothesis is that the majority of these empty rows are a result of no field goals being attempted as the columns with a large amount of NaNs are shooting columns and basketball reference's percentage columns are blank if a player has attempted no shots.

The column towards the end that has 1766 NaNs is the turnover percentage column, it is likely those are for players with limited minutes who don't get the chance to shoot or make turnovers over the course of the game. While it isnt a shooting percentage stat per se, it still uses field goals attempted in its formula and thus if a player produces no field goal attempts he will result in a NaN in the turnover percentage column as well.

In [75]:
players.loc[players['FGA'] > 0].loc[players['FG_PCT'].isna()]

,PLAYER,MP,FG,FGA,FG_PCT,FG(3),FGA(3),FG_PCT(3),FT,FTA,...,WINS,LOSSES,WIN_PCT,GAME_NO,TIME,DATE+TIME,WIN,LOSS,TEAM_STATS_OR_NOT,SEASON


In [76]:
players.loc[players['FGA'] == 0]

,PLAYER,MP,FG,FGA,FG_PCT,FG(3),FGA(3),FG_PCT(3),FT,FTA,...,WINS,LOSSES,WIN_PCT,GAME_NO,TIME,DATE+TIME,WIN,LOSS,TEAM_STATS_OR_NOT,SEASON
10,"Butler,Rasual",0.966667,0,0,NaN,0,0,NaN,0,0,...,1.0,0.0,1.000000,1.0,7:00p,2013-10-29 19:00:00,1,0,0,2014
43,"Mohammed,Nazr",5.266667,0,0,NaN,0,0,NaN,0,0,...,0.0,1.0,0.000000,1.0,8:00p,2013-10-29 20:00:00,1,0,0,2014
78,"Zeller,Tyler",4.000000,0,0,NaN,0,0,NaN,0,0,...,1.0,0.0,1.000000,1.0,7:00p,2013-10-30 19:00:00,1,0,0,2014
142,"Mitchell,Tony",0.316667,0,0,NaN,0,0,NaN,0,0,...,1.0,0.0,1.000000,1.0,7:30p,2013-10-30 19:30:00,1,0,0,2014
143,"Datome,Luigi",0.316667,0,0,NaN,0,0,NaN,0,0,...,1.0,0.0,1.000000,1.0,7:30p,2013-10-30 19:30:00,1,0,0,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56440,"Haslem,Udonis",7.316667,0,0,NaN,0,0,NaN,0,0,...,37.0,45.0,0.451220,82.0,8:00p,2015-04-15 20:00:00,1,0,0,2015
56451,"Green,JaMychal",0.816667,0,0,NaN,0,0,NaN,0,0,...,55.0,27.0,0.670732,82.0,9:30p,2015-04-15 21:30:00,1,0,0,2015
56462,"Sloan,Donald",8.116667,0,0,NaN,0,0,NaN,0,0,...,38.0,44.0,0.463415,82.0,9:30p,2015-04-15 21:30:00,0,1,0,2015
56464,"Rudež,Damjan",0.816667,0,0,NaN,0,0,NaN,0,0,...,38.0,44.0,0.463415,82.0,9:30p,2015-04-15 21:30:00,0,1,0,2015


In [77]:
players.loc[players['TOV_PCT'].isna() == True].loc[players['FGA'] > 0]

,PLAYER,MP,FG,FGA,FG_PCT,FG(3),FGA(3),FG_PCT(3),FT,FTA,...,WINS,LOSSES,WIN_PCT,GAME_NO,TIME,DATE+TIME,WIN,LOSS,TEAM_STATS_OR_NOT,SEASON


In [78]:
players.isna().sum()

PLAYER                   0
MP                       0
FG                       0
FGA                      0
FG_PCT                2586
FG(3)                    0
FGA(3)                   0
FG_PCT(3)            19848
FT                       0
FTA                      0
FT_PCT               21795
ORB                      0
DRB                      0
TRB                      0
AST                      0
STL                      0
BLK                      0
TOV                      0
PF                       0
PTS                      0
+/-                      5
TS_PCT                2239
EFG_PCT               2586
3PA_R                 2586
FT_R                  2586
ORB_PCT                  0
DRB_PCT                  0
TRB_PCT                  0
AST_PCT                  0
STL_PCT                  0
BLK_PCT                  0
TOV_PCT               1766
USG_PCT                  0
O_RTG                    0
D_RTG                    0
B_+/-                    0
GAMEID                   0
T

In [79]:
empty_cols = []
for index,i in players.isna().sum().iteritems():
    if i > 0:
        empty_cols.append(index)

In [80]:
empty_cols

['FG_PCT',
 'FG_PCT(3)',
 'FT_PCT',
 '+/-',
 'TS_PCT',
 'EFG_PCT',
 '3PA_R',
 'FT_R',
 'TOV_PCT']

In [81]:
players.loc[players['+/-'].isna() == True]

,PLAYER,MP,FG,FGA,FG_PCT,FG(3),FGA(3),FG_PCT(3),FT,FTA,...,WINS,LOSSES,WIN_PCT,GAME_NO,TIME,DATE+TIME,WIN,LOSS,TEAM_STATS_OR_NOT,SEASON
6878,"Ellington,Wayne",0.283333,0,0,NaN,0,0,NaN,0,0,...,13.0,8.0,0.619048,21.0,10:00p,2013-12-07 22:00:00,1,0,0,2014
26875,"Temple,Garrett",0.016667,0,0,NaN,0,0,NaN,0,0,...,41.0,38.0,0.518987,79.0,7:00p,2014-04-11 19:00:00,1,0,0,2014
30769,"Jefferson,Cory",0.100000,0,0,NaN,0,0,NaN,0,0,...,4.0,3.0,0.571429,7.0,9:00p,2014-11-12 21:00:00,0,1,0,2015
45030,"Jenkins,John",0.733333,0,0,NaN,0,0,NaN,0,0,...,41.0,9.0,0.820000,50.0,7:30p,2015-02-04 19:30:00,0,1,0,2015
45042,"Butler,Rasual",0.733333,0,0,NaN,0,0,NaN,0,0,...,31.0,19.0,0.620000,50.0,7:30p,2015-02-04 19:30:00,1,0,0,2015


+/- NaNs are likely players who did not register an impact on the court whatsoever, can fill as a 0

In [82]:
players.loc[players['3PA_R'].isna() == True]

,PLAYER,MP,FG,FGA,FG_PCT,FG(3),FGA(3),FG_PCT(3),FT,FTA,...,WINS,LOSSES,WIN_PCT,GAME_NO,TIME,DATE+TIME,WIN,LOSS,TEAM_STATS_OR_NOT,SEASON
10,"Butler,Rasual",0.966667,0,0,NaN,0,0,NaN,0,0,...,1.0,0.0,1.000000,1.0,7:00p,2013-10-29 19:00:00,1,0,0,2014
43,"Mohammed,Nazr",5.266667,0,0,NaN,0,0,NaN,0,0,...,0.0,1.0,0.000000,1.0,8:00p,2013-10-29 20:00:00,1,0,0,2014
78,"Zeller,Tyler",4.000000,0,0,NaN,0,0,NaN,0,0,...,1.0,0.0,1.000000,1.0,7:00p,2013-10-30 19:00:00,1,0,0,2014
142,"Mitchell,Tony",0.316667,0,0,NaN,0,0,NaN,0,0,...,1.0,0.0,1.000000,1.0,7:30p,2013-10-30 19:30:00,1,0,0,2014
143,"Datome,Luigi",0.316667,0,0,NaN,0,0,NaN,0,0,...,1.0,0.0,1.000000,1.0,7:30p,2013-10-30 19:30:00,1,0,0,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56440,"Haslem,Udonis",7.316667,0,0,NaN,0,0,NaN,0,0,...,37.0,45.0,0.451220,82.0,8:00p,2015-04-15 20:00:00,1,0,0,2015
56451,"Green,JaMychal",0.816667,0,0,NaN,0,0,NaN,0,0,...,55.0,27.0,0.670732,82.0,9:30p,2015-04-15 21:30:00,1,0,0,2015
56462,"Sloan,Donald",8.116667,0,0,NaN,0,0,NaN,0,0,...,38.0,44.0,0.463415,82.0,9:30p,2015-04-15 21:30:00,0,1,0,2015
56464,"Rudež,Damjan",0.816667,0,0,NaN,0,0,NaN,0,0,...,38.0,44.0,0.463415,82.0,9:30p,2015-04-15 21:30:00,0,1,0,2015


In [83]:
players.loc[players['3PA_R'].isna() == True].loc[players['FGA'].isna() == 0].loc[players['FT_R'].isna()==True]

,PLAYER,MP,FG,FGA,FG_PCT,FG(3),FGA(3),FG_PCT(3),FT,FTA,...,WINS,LOSSES,WIN_PCT,GAME_NO,TIME,DATE+TIME,WIN,LOSS,TEAM_STATS_OR_NOT,SEASON
10,"Butler,Rasual",0.966667,0,0,NaN,0,0,NaN,0,0,...,1.0,0.0,1.000000,1.0,7:00p,2013-10-29 19:00:00,1,0,0,2014
43,"Mohammed,Nazr",5.266667,0,0,NaN,0,0,NaN,0,0,...,0.0,1.0,0.000000,1.0,8:00p,2013-10-29 20:00:00,1,0,0,2014
78,"Zeller,Tyler",4.000000,0,0,NaN,0,0,NaN,0,0,...,1.0,0.0,1.000000,1.0,7:00p,2013-10-30 19:00:00,1,0,0,2014
142,"Mitchell,Tony",0.316667,0,0,NaN,0,0,NaN,0,0,...,1.0,0.0,1.000000,1.0,7:30p,2013-10-30 19:30:00,1,0,0,2014
143,"Datome,Luigi",0.316667,0,0,NaN,0,0,NaN,0,0,...,1.0,0.0,1.000000,1.0,7:30p,2013-10-30 19:30:00,1,0,0,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56440,"Haslem,Udonis",7.316667,0,0,NaN,0,0,NaN,0,0,...,37.0,45.0,0.451220,82.0,8:00p,2015-04-15 20:00:00,1,0,0,2015
56451,"Green,JaMychal",0.816667,0,0,NaN,0,0,NaN,0,0,...,55.0,27.0,0.670732,82.0,9:30p,2015-04-15 21:30:00,1,0,0,2015
56462,"Sloan,Donald",8.116667,0,0,NaN,0,0,NaN,0,0,...,38.0,44.0,0.463415,82.0,9:30p,2015-04-15 21:30:00,0,1,0,2015
56464,"Rudež,Damjan",0.816667,0,0,NaN,0,0,NaN,0,0,...,38.0,44.0,0.463415,82.0,9:30p,2015-04-15 21:30:00,0,1,0,2015


It's clear, I can impute the null values with 0 as they all come from one of the base box score stats (FGA, TRB, ORB, DRB, FGA(3)] being a 0 or come from a lack of minutes being given to make an impact (+/-)

In [84]:
players.fillna(0, inplace=True)

In [85]:
players.isna().sum()

PLAYER               0
MP                   0
FG                   0
FGA                  0
FG_PCT               0
FG(3)                0
FGA(3)               0
FG_PCT(3)            0
FT                   0
FTA                  0
FT_PCT               0
ORB                  0
DRB                  0
TRB                  0
AST                  0
STL                  0
BLK                  0
TOV                  0
PF                   0
PTS                  0
+/-                  0
TS_PCT               0
EFG_PCT              0
3PA_R                0
FT_R                 0
ORB_PCT              0
DRB_PCT              0
TRB_PCT              0
AST_PCT              0
STL_PCT              0
BLK_PCT              0
TOV_PCT              0
USG_PCT              0
O_RTG                0
D_RTG                0
B_+/-                0
GAMEID               0
TEAM                 0
HOME                 0
AWAY                 0
OPPONENT             0
DATE                 0
WINS                 0
LOSSES     

In [86]:
players.drop(columns=['TIME', 'DATE+TIME', 'TEAM_STATS_OR_NOT'], inplace=True)

In [87]:
players

,PLAYER,MP,FG,FGA,FG_PCT,FG(3),FGA(3),FG_PCT(3),FT,FTA,...,AWAY,OPPONENT,DATE,WINS,LOSSES,WIN_PCT,GAME_NO,WIN,LOSS,SEASON
0,"George,Paul",36.083333,8,16,0.500,3,6,0.500,5,7,...,0.0,ORL,2013-10-29,1.0,0.0,1.000000,1.0,1,0,2014
1,"Stephenson,Lance",35.633333,8,12,0.667,2,3,0.667,1,2,...,0.0,ORL,2013-10-29,1.0,0.0,1.000000,1.0,1,0,2014
2,"West,David",30.716667,4,11,0.364,0,0,0.000,5,5,...,0.0,ORL,2013-10-29,1.0,0.0,1.000000,1.0,1,0,2014
3,"Hill,George",29.066667,2,8,0.250,0,2,0.000,3,4,...,0.0,ORL,2013-10-29,1.0,0.0,1.000000,1.0,1,0,2014
4,"Hibbert,Roy",26.900000,3,6,0.500,0,0,0.000,2,4,...,0.0,ORL,2013-10-29,1.0,0.0,1.000000,1.0,1,0,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56505,"Thompson,Jason",24.733333,5,8,0.625,0,0,0.000,2,4,...,1.0,LAL,2015-04-15,29.0,53.0,0.353659,82.0,0,1,2015
56506,"Williams,Derrick",33.583333,9,16,0.563,2,4,0.500,2,4,...,1.0,LAL,2015-04-15,29.0,53.0,0.353659,82.0,0,1,2015
56507,"Hollins,Ryan",23.266667,5,7,0.714,0,0,0.000,2,7,...,1.0,LAL,2015-04-15,29.0,53.0,0.353659,82.0,0,1,2015
56508,"Stockton,David",19.383333,2,3,0.667,1,1,1.000,0,1,...,1.0,LAL,2015-04-15,29.0,53.0,0.353659,82.0,0,1,2015


In [88]:
players.columns

Index(['PLAYER', 'MP', 'FG', 'FGA', 'FG_PCT', 'FG(3)', 'FGA(3)', 'FG_PCT(3)',
       'FT', 'FTA', 'FT_PCT', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV',
       'PF', 'PTS', '+/-', 'TS_PCT', 'EFG_PCT', '3PA_R', 'FT_R', 'ORB_PCT',
       'DRB_PCT', 'TRB_PCT', 'AST_PCT', 'STL_PCT', 'BLK_PCT', 'TOV_PCT',
       'USG_PCT', 'O_RTG', 'D_RTG', 'B_+/-', 'GAMEID', 'TEAM', 'HOME', 'AWAY',
       'OPPONENT', 'DATE', 'WINS', 'LOSSES', 'WIN_PCT', 'GAME_NO', 'WIN',
       'LOSS', 'SEASON'],
      dtype='object')

In [89]:
players['FG(3)_MISSED'] = players['FGA(3)'] - players['FG(3)']
players['FG_MISSED'] = players['FGA'] - players['FG']
players['FG(2)'] = players['FG']-players['FG(3)']
players['FGA(2)'] = players['FGA']-players['FGA(3)']
players['FG(2)_MISSED'] = players['FG_MISSED']-players['FG(3)_MISSED']

In [90]:
players

,PLAYER,MP,FG,FGA,FG_PCT,FG(3),FGA(3),FG_PCT(3),FT,FTA,...,WIN_PCT,GAME_NO,WIN,LOSS,SEASON,FG(3)_MISSED,FG_MISSED,FG(2),FGA(2),FG(2)_MISSED
0,"George,Paul",36.083333,8,16,0.500,3,6,0.500,5,7,...,1.000000,1.0,1,0,2014,3,8,5,10,5
1,"Stephenson,Lance",35.633333,8,12,0.667,2,3,0.667,1,2,...,1.000000,1.0,1,0,2014,1,4,6,9,3
2,"West,David",30.716667,4,11,0.364,0,0,0.000,5,5,...,1.000000,1.0,1,0,2014,0,7,4,11,7
3,"Hill,George",29.066667,2,8,0.250,0,2,0.000,3,4,...,1.000000,1.0,1,0,2014,2,6,2,6,4
4,"Hibbert,Roy",26.900000,3,6,0.500,0,0,0.000,2,4,...,1.000000,1.0,1,0,2014,0,3,3,6,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56505,"Thompson,Jason",24.733333,5,8,0.625,0,0,0.000,2,4,...,0.353659,82.0,0,1,2015,0,3,5,8,3
56506,"Williams,Derrick",33.583333,9,16,0.563,2,4,0.500,2,4,...,0.353659,82.0,0,1,2015,2,7,7,12,5
56507,"Hollins,Ryan",23.266667,5,7,0.714,0,0,0.000,2,7,...,0.353659,82.0,0,1,2015,0,2,5,7,2
56508,"Stockton,David",19.383333,2,3,0.667,1,1,1.000,0,1,...,0.353659,82.0,0,1,2015,0,1,1,2,1


In [91]:
games_14

,TEAM_HOME,TEAM_AWAY,FG_HOME,FG_AWAY,FGA_HOME,FGA_AWAY,FG_PCT_HOME,FG_PCT_AWAY,FG(3)_HOME,FG(3)_AWAY,...,FG(2)_HOME,FG(2)_AWAY,FGA(2)_HOME,FGA(2)_AWAY,FG(2)_MISSED_HOME,FG(2)_MISSED_AWAY,GAMEID,DATE,SEASON,OTs
0,IND,ORL,34,36,71,93,0.479,0.387,7,9,...,27,27,54,74,27,47,201310290IND,2013-10-29,2014,0
1,MIA,CHI,37,35,72,83,0.514,0.422,11,7,...,26,28,52,57,26,29,201310290MIA,2013-10-29,2014,0
2,LAL,LAC,42,41,93,83,0.452,0.494,14,8,...,28,33,64,62,36,29,201310290LAL,2013-10-29,2014,0
3,CLE,BRK,35,33,84,82,0.417,0.402,5,9,...,30,24,69,58,39,34,201310300CLE,2013-10-30,2014,0
4,TOR,BOS,38,32,86,66,0.442,0.485,5,3,...,33,29,69,53,36,24,201310300TOR,2013-10-30,2014,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,NYK,DET,33,41,77,81,0.429,0.506,7,10,...,26,31,58,55,32,24,201504150NYK,2015-04-15,2015,0
2456,PHI,MIA,38,39,83,89,0.458,0.438,5,12,...,33,27,54,58,21,31,201504150PHI,2015-04-15,2015,0
2457,MEM,IND,36,32,76,78,0.474,0.410,2,8,...,34,24,67,54,33,30,201504150MEM,2015-04-15,2015,0
2458,GSW,DEN,47,45,90,83,0.522,0.542,13,6,...,34,39,62,65,28,26,201504150GSW,2015-04-15,2015,0


In [92]:
ots = games_14[['GAMEID','OTs']]

In [93]:
players = pd.merge(players,ots, how='inner', on='GAMEID').copy()

In [94]:
gen_cols = ['OTs','GAMEID', 'SEASON', 'DATE']
stat_cols = []
for col in players.columns:
    if col not in gen_cols:
        stat_cols.append(col)

In [95]:
stat_cols

['PLAYER',
 'MP',
 'FG',
 'FGA',
 'FG_PCT',
 'FG(3)',
 'FGA(3)',
 'FG_PCT(3)',
 'FT',
 'FTA',
 'FT_PCT',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 '+/-',
 'TS_PCT',
 'EFG_PCT',
 '3PA_R',
 'FT_R',
 'ORB_PCT',
 'DRB_PCT',
 'TRB_PCT',
 'AST_PCT',
 'STL_PCT',
 'BLK_PCT',
 'TOV_PCT',
 'USG_PCT',
 'O_RTG',
 'D_RTG',
 'B_+/-',
 'TEAM',
 'HOME',
 'AWAY',
 'OPPONENT',
 'WINS',
 'LOSSES',
 'WIN_PCT',
 'GAME_NO',
 'WIN',
 'LOSS',
 'FG(3)_MISSED',
 'FG_MISSED',
 'FG(2)',
 'FGA(2)',
 'FG(2)_MISSED']

In [96]:
players = players[stat_cols + gen_cols].copy()

In [97]:
players

,PLAYER,MP,FG,FGA,FG_PCT,FG(3),FGA(3),FG_PCT(3),FT,FTA,...,LOSS,FG(3)_MISSED,FG_MISSED,FG(2),FGA(2),FG(2)_MISSED,OTs,GAMEID,SEASON,DATE
0,"George,Paul",36.083333,8,16,0.500,3,6,0.500,5,7,...,0,3,8,5,10,5,0,201310290IND,2014,2013-10-29
1,"Stephenson,Lance",35.633333,8,12,0.667,2,3,0.667,1,2,...,0,1,4,6,9,3,0,201310290IND,2014,2013-10-29
2,"West,David",30.716667,4,11,0.364,0,0,0.000,5,5,...,0,0,7,4,11,7,0,201310290IND,2014,2013-10-29
3,"Hill,George",29.066667,2,8,0.250,0,2,0.000,3,4,...,0,2,6,2,6,4,0,201310290IND,2014,2013-10-29
4,"Hibbert,Roy",26.900000,3,6,0.500,0,0,0.000,2,4,...,0,0,3,3,6,3,0,201310290IND,2014,2013-10-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51586,"Thompson,Jason",24.733333,5,8,0.625,0,0,0.000,2,4,...,1,0,3,5,8,3,0,201504150LAL,2015,2015-04-15
51587,"Williams,Derrick",33.583333,9,16,0.563,2,4,0.500,2,4,...,1,2,7,7,12,5,0,201504150LAL,2015,2015-04-15
51588,"Hollins,Ryan",23.266667,5,7,0.714,0,0,0.000,2,7,...,1,0,2,5,7,2,0,201504150LAL,2015,2015-04-15
51589,"Stockton,David",19.383333,2,3,0.667,1,1,1.000,0,1,...,1,0,1,1,2,1,0,201504150LAL,2015,2015-04-15


In [98]:
players['DATE'] = pd.to_datetime(players['DATE'])

In [99]:
players['TEAM'] = players['TEAM'].map(lambda x:'CHA' if x == 'CHO' else x)

In [100]:
players.to_csv('./cleaned_data/player_data/cleaned_players_14_15.csv')

In [101]:
players.columns

Index(['PLAYER', 'MP', 'FG', 'FGA', 'FG_PCT', 'FG(3)', 'FGA(3)', 'FG_PCT(3)',
       'FT', 'FTA', 'FT_PCT', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV',
       'PF', 'PTS', '+/-', 'TS_PCT', 'EFG_PCT', '3PA_R', 'FT_R', 'ORB_PCT',
       'DRB_PCT', 'TRB_PCT', 'AST_PCT', 'STL_PCT', 'BLK_PCT', 'TOV_PCT',
       'USG_PCT', 'O_RTG', 'D_RTG', 'B_+/-', 'TEAM', 'HOME', 'AWAY',
       'OPPONENT', 'WINS', 'LOSSES', 'WIN_PCT', 'GAME_NO', 'WIN', 'LOSS',
       'FG(3)_MISSED', 'FG_MISSED', 'FG(2)', 'FGA(2)', 'FG(2)_MISSED', 'OTs',
       'GAMEID', 'SEASON', 'DATE'],
      dtype='object')

In [102]:
len(players.columns)

55

In [103]:
# CREATING A PLAYER CLEANING FUNCTION SIMILAR TO TEAM STATS CLEANING FUNCTION
def clean_to_player(df):
    games = clean_df_to_games(df)
    players = df.loc[df['TEAM_STATS_OR_NOT'] == 0].copy()
    players.fillna(0)
    players['FG(3)_MISSED'] = players['FGA(3)'] - players['FG(3)']
    players['FG_MISSED'] = players['FGA'] - players['FG']
    players['FG(2)'] = players['FG']-players['FG(3)']
    players['FGA(2)'] = players['FGA']-players['FGA(3)']
    players['FG(2)_MISSED'] = players['FG_MISSED']-players['FG(3)_MISSED']
    players.drop(columns=['TIME', 'DATE+TIME', 'TEAM_STATS_OR_NOT'], inplace=True)
    ots = games[['GAMEID','OTs']]
    players = pd.merge(players,ots, how='inner', on='GAMEID').copy()
    players['DATE'] = pd.to_datetime(players['DATE'])
    gen_cols = ['OTs','GAMEID', 'SEASON', 'DATE']
    stat_cols = []
    for col in players.columns:
        if col not in gen_cols:
            stat_cols.append(col)
    players = players[stat_cols + gen_cols].copy()
    return players

In [104]:
players_16 = clean_to_player(df_16)

In [105]:
players_16

,PLAYER,MP,FG,FGA,FG_PCT,FG(3),FGA(3),FG_PCT(3),FT,FTA,...,LOSS,FG(3)_MISSED,FG_MISSED,FG(2),FGA(2),FG(2)_MISSED,OTs,GAMEID,SEASON,DATE
0,"Butler,Jimmy",36.633333,6,14,0.429,0,2,0.00,5,5,...,0,2,8,6,12,6,0,201510270CHI,2016,2015-10-27
1,"Rose,Derrick",32.466667,8,22,0.364,0,2,0.00,2,4,...,0,2,14,8,20,12,0,201510270CHI,2016,2015-10-27
2,"Gasol,Pau",31.550000,1,7,0.143,0,1,0.00,0,0,...,0,1,6,1,6,5,0,201510270CHI,2016,2015-10-27
3,"Snell,Tony",28.800000,4,9,0.444,2,5,0.40,1,2,...,0,3,5,2,4,2,0,201510270CHI,2016,2015-10-27
4,"Mirotić,Nikola",25.233333,6,11,0.545,3,4,0.75,4,4,...,0,1,5,3,7,4,0,201510270CHI,2016,2015-10-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52199,"Crawford,Jordan",22.633333,6,12,0.500,3,4,0.75,0,0,...,0,1,6,3,8,5,0,201704120POR,2017,2017-04-12
52200,"Toupane,Axel",20.883333,4,4,1.000,0,0,NaN,0,0,...,0,0,0,4,4,0,0,201704120POR,2017,2017-04-12
52201,"Cook,Quinn",19.666667,4,7,0.571,1,1,1.00,2,2,...,0,0,3,3,6,3,0,201704120POR,2017,2017-04-12
52202,"Motiejūnas,Donatas",16.850000,3,11,0.273,0,4,0.00,0,1,...,0,4,8,3,7,4,0,201704120POR,2017,2017-04-12


In [106]:
players_16.columns

Index(['PLAYER', 'MP', 'FG', 'FGA', 'FG_PCT', 'FG(3)', 'FGA(3)', 'FG_PCT(3)',
       'FT', 'FTA', 'FT_PCT', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV',
       'PF', 'PTS', '+/-', 'TS_PCT', 'EFG_PCT', '3PA_R', 'FT_R', 'ORB_PCT',
       'DRB_PCT', 'TRB_PCT', 'AST_PCT', 'STL_PCT', 'BLK_PCT', 'TOV_PCT',
       'USG_PCT', 'O_RTG', 'D_RTG', 'B_+/-', 'TEAM', 'HOME', 'AWAY',
       'OPPONENT', 'WINS', 'LOSSES', 'WIN_PCT', 'GAME_NO', 'WIN', 'LOSS',
       'FG(3)_MISSED', 'FG_MISSED', 'FG(2)', 'FGA(2)', 'FG(2)_MISSED', 'OTs',
       'GAMEID', 'SEASON', 'DATE'],
      dtype='object')

In [107]:
players_16.to_csv('./cleaned_data/player_data/cleaned_players_16_17.csv')

In [108]:
players_18 = clean_to_player(df_18)

In [109]:
players_18

,PLAYER,MP,FG,FGA,FG_PCT,FG(3),FGA(3),FG_PCT(3),FT,FTA,...,LOSS,FG(3)_MISSED,FG_MISSED,FG(2),FGA(2),FG(2)_MISSED,OTs,GAMEID,SEASON,DATE
0,"James,LeBron",41.200000,12,19,0.632,1,5,0.200,4,4,...,1,4,7,11,14,3,0,201710170CLE,2018,2017-10-17
1,"Crowder,Jae",34.733333,3,10,0.300,1,5,0.200,4,4,...,1,4,7,2,5,3,0,201710170CLE,2018,2017-10-17
2,"Rose,Derrick",31.250000,5,14,0.357,1,3,0.333,3,4,...,1,2,9,4,11,7,0,201710170CLE,2018,2017-10-17
3,"Wade,Dwyane",28.500000,3,10,0.300,0,1,0.000,2,2,...,1,1,7,3,9,6,0,201710170CLE,2018,2017-10-17
4,"Love,Kevin",28.400000,4,9,0.444,1,4,0.250,6,7,...,1,3,5,3,5,2,0,201710170CLE,2018,2017-10-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52188,"Brewer,Corey",23.450000,0,0,NaN,0,0,NaN,0,0,...,1,0,0,0,0,0,0,201904100POR,2019,2019-04-10
52189,"Mason,Frank",17.933333,6,13,0.462,2,6,0.333,1,2,...,1,4,7,4,7,3,0,201904100POR,2019,2019-04-10
52190,"Swanigan,Caleb",17.583333,2,5,0.400,0,1,0.000,0,0,...,1,1,3,2,4,2,0,201904100POR,2019,2019-04-10
52191,"Williams,Troy",11.033333,2,5,0.400,0,1,0.000,0,0,...,1,1,3,2,4,2,0,201904100POR,2019,2019-04-10


In [110]:
players_18.to_csv('./cleaned_data/player_data/cleaned_players_18_19.csv')

In [111]:
pd.concat([players, players_16, players_18]).to_csv('./cleaned_data/player_data/cleaned_players_full.csv')

In [112]:
pd.concat([games_14,games_16,games_18]).to_csv('./cleaned_data/team_data/cleaned_games_full.csv')